# Keras を使ったモデルの学習と推論を SageMaker で行う

#### ノートブックに含まれる内容

- Keras API を SageMaker で使うときの，基本的なやりかた
- `tf.keras.applications` をベースに Keras のコンパイル済みモデルを SageMaker で使うときの説明

#### ノートブックで使われている手法の詳細

- アルゴリズム: DNN
- データ: Abalone

## セットアップ

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## データのロード

ローカルのデータを S3 にアップロードします．使用する [Abalone](https://en.wikipedia.org/wiki/Abalone) データは，貝殻の長さや高さ，重さといったパラメタをもとに，アワビの年齢を推測するというものになります．[Abalone データセット](https://archive.ics.uci.edu/ml/datasets/Abalone)には以下の[特徴量](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.names)が含まれています．

| Feature        | Description                                               |
| -------------- | --------------------------------------------------------- |
| Length         | 貝殻の長さ (最もと長い方向で測った時，単位は mm)                 |
| Diameter       | 貝殻の直径 (単位は mm)                                       |
| Height         | 貝殻の高さ (中身が入っている時の高さで，単位は mm)               |
| Whole Weight   | 貝全体の重さ (単位はグラム)                                   |
| Shucked Weight | アワビ本体だけの重さ (単位はグラム)                            |
| Viscera Weight | 血を抜いた後の内臓の重さ (単位はグラム)                         |
| Shell Weight   | 貝殻自体の重さ (単位はグラム)                                 |

予測対象のラベルデータは，貝殻の成長線の数で，これは年齢の近似値になります．以下を実行する前に，**<span style="color: red;"> `data/あabalone/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/abalone/XX')

## Keras を使ったスクリプトの中身を確認

`tf.keras.layers` 等を使ってモデルを組むだけであれば，基本的には Tensorflow のときと違いはほとんどありません．ネットワークの記述を行って以降は，Tensorflow で損失関数やオプティマイザ等を記述する形になります．

また，`tf.keras.applications` など，既存のモデルをそのまま使う場合には，以下のように `keras_model_fn` でコンパイル済みモデルを記述します．詳細は[こちら](https://github.com/aws/sagemaker-python-sdk#using-a-keras-model-instead-of-a-model_fn)をご覧ください．

```python
def keras_model_fn(hyperparameters):
  # Instantiate a Keras inception v3 model.
  keras_inception_v3 = tf.keras.applications.inception_v3.InceptionV3(weights=None)
  # Compile model with the optimizer, loss, and metrics you'd like to train with.
  keras_inception_v3.compile(optimizer=tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9),
                        loss='categorical_crossentropy', metric='accuracy')
  return keras_inception_v3
```

実際に実行するスクリプトの中身を確認しましょう．

In [ ]:
!cat 'abalone.py'

## モデルの学習を実行

以降は，通常のTensorflow の学習・推論プロセスと同様です．

In [ ]:
from sagemaker.tensorflow import TensorFlow

abalone_estimator = TensorFlow(entry_point='abalone.py',
                               role=role,
                               training_steps= 100,                                  
                               evaluation_steps= 100,
                               hyperparameters={'learning_rate': 0.001},
                               train_instance_count=1,
                               train_instance_type='ml.m4.xlarge')

abalone_estimator.fit(inputs)

## モデルの推論を実行

`deploy()` メソッドを使って，学習済みモデルのデプロイを実施します．それが完了したら，`predict()` メソッドで実際に予測を行ってみます．

In [ ]:
abalone_predictor = abalone_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
import tensorflow as tf
import numpy as np

# ローカルのデータを読み込んで，1 行を protbuf にエンコードしてリクエストを投げる
prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=os.path.join('data/abalone_predict.csv'), target_dtype=np.int, features_dtype=np.float32)

data = prediction_set.data[0]
tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)

In [ ]:
abalone_predictor.predict(tensor_proto)

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker.Session().delete_endpoint(abalone_predictor.endpoint)